DOCUMENT SEARCH ENGINE:

In [1]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words
from nltk import punkt

import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math

In [2]:
#constant value alpha used during TF-IDF ranking.
alpha = 0.3

In [3]:
folders = [x[0] for x in os.walk(str(os.getcwd())+'/data/stories/')]
folders[0] = folders[0][:len(folders[0])-1]

In [4]:
folders

['d:\\PYTHON\\SEARCH ENGINE PROJECT_2/data/stories',
 'd:\\PYTHON\\SEARCH ENGINE PROJECT_2/data/stories/FARNON',
 'd:\\PYTHON\\SEARCH ENGINE PROJECT_2/data/stories/SRE']

In [5]:
dataset = []

c= False

for i in folders:
    file = open(i + "/index.html", 'r')
    text = file.read().strip()
    #strip remove all the leading and trailing spaces
    file.close()

    #We can clearly observe that each file name is enclosed between (><A HREF=”) and (”) and each title is between (<BR><TD>) and (\n)
    file_name = re.findall('><A HREF="(.*)">', text)
    file_title = re.findall('<BR><TD> (.*)\n', text)
    
    if c == False:
        file_name = file_name[2:]
        c = True
        #only for root as it has 2 folders
        
    print(len(file_name), len(file_title))
    #because the root has 2 folders and we shouldn't consider the two folders
    #3 outputs for 3 Folders


    for j in range(len(file_name)):
        dataset.append((i + "/" + file_name[j], file_title[j]))

452 452
0 0
15 15


In [6]:
dataset

[('d:\\PYTHON\\SEARCH ENGINE PROJECT_2/data/stories/100west.txt',
  'Going 100 West by 53 North by Jim Prentice (1990)'),
 ('d:\\PYTHON\\SEARCH ENGINE PROJECT_2/data/stories/13chil.txt',
  'The Story of the Sly Fox'),
 ('d:\\PYTHON\\SEARCH ENGINE PROJECT_2/data/stories/14.lws',
  'A Smart Bomb with a Language Parser'),
 ('d:\\PYTHON\\SEARCH ENGINE PROJECT_2/data/stories/16.lws',
  'Two Guys in a Garage, by M. Pshota'),
 ('d:\\PYTHON\\SEARCH ENGINE PROJECT_2/data/stories/17.lws',
  'The Early Days of a High-Tech Start-up are Magic (November 18, 1991) by M. Peshota'),
 ('d:\\PYTHON\\SEARCH ENGINE PROJECT_2/data/stories/18.lws',
  'The Couch, the File Cabinet, and the Calendar, by M. Peshota (December 9, 1991)'),
 ('d:\\PYTHON\\SEARCH ENGINE PROJECT_2/data/stories/19.lws',
  'Engineering the Future of American Technology by M. Peshota (January 5, 1992)'),
 ('d:\\PYTHON\\SEARCH ENGINE PROJECT_2/data/stories/20.lws',
  'What Research and Development Was Always Meant to Be, by M. Peshota'),


In [7]:
len(dataset)

467

In [8]:
N = len(dataset)

In [9]:
#In our problem statement, 
#we have very different types of documents,
#this can cause few errors in reading the documents due to encoding compatibility. 
#use encoding=”utf8", errors=’ignore’ in the open() method.
def print_doc(id):
    print(dataset[id])
    file = open(dataset[id][0], 'r', encoding='cp1250')
    text = file.read().strip()
    file.close()
    print(text)

In [53]:
def print_title(id):
    print(dataset[id][1])

PRE PROCESSING 101

In [11]:
def convert_lower_case(data):
    return np.char.lower(data)

In [12]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [13]:
def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text
#return the new_text with no single tokens or where length of word is > 1.

In [14]:
#practice 1
data2 = "Antidises@tablishm,,en#tari%,""nism Jay's Machine's,   will, death."
symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
for i in range(len(symbols)):
    data2 = np.char.replace(data2, symbols[i], ' ')
    data2 = np.char.replace(data2, "  ", " ")
data2 = np.char.replace(data2, ',', '') #because it doesn't get replaced with space
print(data2)

Antidises tablishmen tari nism Jay's Machine's will death 


In [15]:
def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

In [16]:
def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

In [17]:
#practice 2
# import SyllableTokenizer() method from nltk                   
from nltk import word_tokenize
from nltk import SyllableTokenizer
# Create a reference variable for Class word_tokenize
tk = SyllableTokenizer()
variable_1 = "Antidisestablishmentarianism"    
# Use tokenize method
variable_1 = tk.tokenize(variable_1)
print(variable_1)

['Anti', 'di', 'ses', 'ta', 'blis', 'hmen', 'ta', 'ria', 'nism']


In [18]:
#practice 3
stemmer = PorterStemmer()
print(stemmer.stem("swimming"))
print(stemmer.stem("Living"))
print(word_tokenize) 
#word_tokenizer is a library of nltk, package of punkt.

swim
live
<function word_tokenize at 0x0000015AE407EC10>


In [19]:
def stemming(data):
    stemmer= PorterStemmer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return new_text


In [20]:
def convert_numbers(data):
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        try:
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
    new_text = np.char.replace(new_text, "-", " ")
    return new_text

PRE PROCESSING 102

In [21]:
def preprocess(data):
    data = convert_lower_case(data)
    data = remove_punctuation(data) #remove comma seperately
    data = remove_apostrophe(data)
    data = remove_stop_words(data)
    data = convert_numbers(data)
    data = stemming(data)
    data = remove_punctuation(data)
    data = convert_numbers(data)
    data = stemming(data) #needed again as we need to stem the words
    data = remove_punctuation(data) #needed again as num2word is giving few hypens and commas fourty-one
    data = remove_stop_words(data) #needed again as num2word is giving stop words 101 - one hundred and one
    return data

Extracting Data

In [22]:
#we have very different types of documents, this can cause few errors in reading the documents due to encoding compatibility. 
#to resolve this, just use encoding=”utf8", errors=’ignore’ in the open() method.
processed_text = []
processed_title = []

#UTF-8 is a byte oriented encoding
#UTF-8 is one of the most commonly used encodings, and Python often defaults to using it.

for i in dataset[:N]:
    file = open(i[0], 'r', encoding="utf8", errors='ignore')
    text = file.read().strip()
    file.close()

    processed_text.append(word_tokenize(str(preprocess(text))))
    processed_title.append(word_tokenize(str(preprocess(i[1]))))

Calculate DF for all words

In [23]:
DF = {}
#dictionary to store word and their indexes/DF
#processed text is the body of the text 
#we should also calculate the title
#title has different weight over body of the text, but the process remains same.

for i in range(N):
    tokens = processed_text[i]  #body
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}

    tokens = processed_title[i] #title
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}
for i in DF:
    DF[i] = len(DF[i])
# print(DF) - to print no of occurrences of words in the entire document ~ DF

#DF

In [24]:
total_vocab_size = len(DF)
total_vocab_size

32350

In [25]:
total_vocab = [x for x in DF]
print(total_vocab[0:20]) #first 20 keys

['sharewar', 'trial', 'project', 'freewar', 'need', 'support', 'continu', 'one', 'hundr', 'west', 'fifti', 'three', 'north', 'jim', 'prentic', 'copyright', 'thousand', 'nine', 'nineti', 'brandon']


In [26]:
def doc_freq(word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c

Calculating TF-IDF for body, we will consider this as the actual tf-idf as we will add the title weight to this.

In [27]:
#practice 4
# import counter class from collections module
from collections import Counter
# string as an iterable data container
x = Counter("you are smart but are you smart enough")
# printing the elements of counter object
for i in x.elements(): 
    #.elements() is a str iterator
    print ( i, end = "")

yyooouuuu       aaaarrrreeessmmtttbngh

In [28]:
#practice 5 
Counter(list("mississippi is a jerk"))

Counter({'m': 1,
         'i': 5,
         's': 5,
         'p': 2,
         ' ': 3,
         'a': 1,
         'j': 1,
         'e': 1,
         'r': 1,
         'k': 1})

In [29]:
#practice 6
# print(processed_title[i])
tokens = processed_text[4]
# print(Counter(tokens + processed_title[4]))
print(Counter(processed_title[1]))

Counter({'stori': 1, 'sli': 1, 'fox': 1})


In [30]:
doc = 0

tf_idf = {}

#N = len(dataset)
for i in range(N):

    tokens = processed_text[i] 
    #tokens is found in each document body
    counter = Counter(tokens + processed_title[i]) 
    #counts each token/word
    words_counts = len(tokens + processed_title[i])

    for token in np.unique(tokens):

        tf = counter[token]/words_counts
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1))      #numerator is added 1 to avoid negative values

        tf_idf[doc, token] = tf*idf
    
    doc+=1

Calculating TF-IDF for Title

In [31]:
doc = 0

tf_idf_title = {}

for i in range(N):
    
    tokens = processed_title[i]
    #tokens is found in each document title
    counter = Counter(tokens + processed_text[i])
    words_count = len(tokens + processed_text[i])

    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1))   #numerator is added 1 to avoid negative values
        
        tf_idf_title[doc, token] = tf*idf

    doc += 1

# tf_idf_title

In [32]:
tf_idf[(0,'go')]

0.0002906893990853149

In [33]:
tf_idf_title[(0,"go")]

0.0002906893990853149

In [34]:
len(tf_idf)

344095

In [35]:
#practice 7
#printing the tf idf score of tf_idf_title of first x keys, by converting it to list
for key in list(tf_idf_title)[:5]:
    print(key)
    print(tf_idf_title[key])

(0, 'fifti')
0.005434960598980563
(0, 'go')
0.0002906893990853149
(0, 'hundr')
0.002570392381970895
(0, 'jim')
0.005269857144642146
(0, 'nine')
0.0008420698058556812


MERGING the TF-IDF, according to weights.

In [36]:
for i in tf_idf:
    tf_idf[i] *= alpha   

In [37]:
for i in tf_idf_title:
    tf_idf[i] = tf_idf_title[i]

In [38]:
len(tf_idf)

344378

In [39]:
with open("pickles/pickle1.pickle", 'wb') as f:
    pickle.dump(tf_idf,f)

In [40]:
# directly load here
with open("pickles/pickle1.pickle", 'rb') as f:
    pickle.load(f)

TF-IDF Matching Score Ranking

In [65]:
#k is the number documents that will shown as results
def matching_score(k, query):
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))

    print("Matching Score")
    print("\nQuery: ", query)
    print("")
    print(tokens)

    query_weights ={}

    #when we find a token that is in the query 
    # we will add the document id to another dictionary along with the tf-idf value
    #(document,token)key in tf_idf so key[1]
    for key in tf_idf:
        if key[1] in tokens:   
            try:
                query_weights[key[0]] += tf_idf[key]
                #(document ID's,tf-idf) pair
            except:
                query_weights[key[0]] = tf_idf[key]
    # print(query_weights)

    query_weights = sorted(query_weights.items(), key=lambda x: x[1], reverse=True)
    #reverse true means sorted array is reversed OR is sorted in descending order.

    print("")
    
    l = []
    
    for i in query_weights[:k]:
        l.append(i[0])
    
    print("doc ID's: ",l)

    print("Title of the Document: ")
    for i in range(0,len(l)):
        # print(l[i])
        print_title(l[i])

matching_score(5,"Last night when I went to bed I was feeling a little weird") #5 = no of documents for search

Matching Score

Query:  Last night when I went to bed I was feeling a little weird

['last', 'night', 'went', 'bed', 'feel', 'littl', 'weird']

doc ID's:  [292, 352, 272, 269, 9]
Title of the Document: 
Someone Talks About Some Weird Dreams He Has
Little White Lines, by Joe Raymundo
The Story of Little Red Riding Hood
The Little Matchgirl
The Story of the 3 Little Pigs


In [54]:
print_title(352)
# print_doc(352)

Little White Lines, by Joe Raymundo


TF-IDF Cosine Similarity Ranking

In [66]:
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

In [68]:
#practice_8
#use of numpy.zeroes
print(np.zeros((5,4), dtype=float))
s = (2,2)
np.zeros(s)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


array([[0., 0.],
       [0., 0.]])

Vectorising tf-idf

In [70]:
#total_vocab_size = len(DF)
#total_vocab = [x for x in DF] contains all the words i.e. DF
D = np.zeros((N, total_vocab_size))
for i in tf_idf:
    try:
        ind = total_vocab.index(i[1])
        D[i[0]][ind] = tf_idf[i]
    except:
        pass

In [72]:
def gen_vector(tokens):

    Q = np.zeros((len(total_vocab)))
    
    counter = Counter(tokens)
    words_count = len(tokens)

    query_weights = {}
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = math.log((N+1)/(df+1))

        try:
            ind = total_vocab.index(token)
            Q[ind] = tf*idf
        except:
            pass
    return Q

In [77]:
def cosine_similarity(k, query):
    print("Cosine Similarity")
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))
    
    print("\nQuery:", query)
    print("")
    print(tokens)
    
    d_cosines = []
    
    query_vector = gen_vector(tokens)
    
    for d in D:
        d_cosines.append(cosine_sim(query_vector, d))
        
    out = np.array(d_cosines).argsort()[-k:][::-1]
    
    print("")
    
    print(out)

    for i in out:
        print(i, dataset[i][1]) #doc ID and title

In [79]:
Q = cosine_similarity(5,"lovers")

Cosine Similarity

Query: lovers

['lover']

[ 83 293  51  85 116]
83 A Story for Cat Lovers
293 Poem: Through My Eyes
51 The Adventures of Bert and Bernece, by  Francis U. Kaltenbaugh
85 Story: Iron Man
116 Consumated Dreams, by Melanie Martin Vessels


In [80]:
print_title(83)

A Story for Cat Lovers


In [81]:
print_doc(83)

('d:\\PYTHON\\SEARCH ENGINE PROJECT_2/data/stories/bulfelis.txt', 'A Story for Cat Lovers')
Msg: #3085  Sec: 3 - TALL Tales
      07-Mar-89  03:58 PM
Subj: The Felis-Humano Wars
From: Thief Killer
  To: Cat Lovers

     I crouched down in the shadow of the counter between the kitchen
 and the dinette and quickly assessed my current situation. Not bad consid-
 ering. I had my knife, two throwing spikes and one flechette round left in
 my pistol, and I counted no more than three of the enemy left mobile out
 there. Damn cats had been ready to mount a major offensive, but I had
 caught them a little flat footed. They weren't expecting anyone to come
 by the mobile home where my friend, Disk Driver, lived. Shaggra! I hadn't
 expected to come by either, but since I'd happened to be in the neighbor-
 hood...you know how the rest goes.
     I pulled up noticing that the lights were on and music playing
 loudly
 over Driver's stereo. The high volume was typical, but it didn't strike
 me as odd